In [ ]:
import torch
import torchvision 
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms, datasets 

In [ ]:
train = datasets.FashionMNIST("",train=True,download=True,transform=transforms.Compose([transforms.ToTensor()])) 
test = datasets.FashionMNIST("",train=False,download=True,transform=transforms.Compose([transforms.ToTensor()])) 

Extracting FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMNIST/raw


Extracting FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMNIST/raw



Extracting FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMNIST/raw


Extracting FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [ ]:
trainloader = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testloader = torch.utils.data.DataLoader(test, batch_size=10, shuffle=True)

In [ ]:
train[0][0].size()

torch.Size([1, 28, 28])

In [ ]:
# BUILDING CNN MODEL
class CNN(nn.Module):                                   
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,stride=1)
    self.conv2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1)
    self.pool = nn.MaxPool2d(2,2)                             
    self.fc1 = nn.Linear(256,128)
    self.dropout = nn.Dropout(0.2)    
    self.fc2 = nn.Linear(128,64)
    self.fc3 = nn.Linear(64,10)  
  
  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(len(x),256)
    x = F.relu(self.fc1(x))
    x = self.dropout(F.relu(self.fc2(x)))
    x = F.log_softmax(self.fc3(x))
    return x

model = CNN()
print(model)


CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [ ]:
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
epochs = 10
for e in range(epochs):
  for data in trainloader:
    X,y = data
    model.zero_grad()
    output = model(X)
    loss = F.nll_loss(output,y)
    loss.backward()
    optimizer.step()
  print(loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.0837, grad_fn=<NllLossBackward>)
tensor(0.3946, grad_fn=<NllLossBackward>)
tensor(0.8881, grad_fn=<NllLossBackward>)
tensor(0.3745, grad_fn=<NllLossBackward>)
tensor(0.3732, grad_fn=<NllLossBackward>)
tensor(0.5071, grad_fn=<NllLossBackward>)
tensor(0.1536, grad_fn=<NllLossBackward>)
tensor(0.4850, grad_fn=<NllLossBackward>)
tensor(0.0929, grad_fn=<NllLossBackward>)
tensor(0.0534, grad_fn=<NllLossBackward>)


In [ ]:
correct = 0
total = 0
for data in trainloader:
  X,y = data
  output = model(X)
  for idx,i in enumerate(output):
    if torch.argmax(i) == y[idx]:
      correct += 1
    total += 1
print(f'Accuracy: {round(correct*100/total,3)}')  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy: 91.743


In [ ]:
correct = 0
total = 0
for data in testloader:
  X,y = data
  output = model(X)
  for idx,i in enumerate(output):
    if torch.argmax(i) == y[idx]:
      correct += 1
    total += 1
print(f'Accuracy: {round(correct*100/total,3)}')  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy: 89.43
